model can train with seasons () to ()
what are the features?


dataframe:
home_team | away_team | ranking_home_team | ranking_away_team || result
________________________________________________________________________

4 primeres columnes: features
5 columna: target

train (season tal to tal: info features + target)

test (apply to () with teh following features)

print:

features, target, predictions